In [120]:
#1 Write a python program to display all the header tags from wikipedia.org.
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [121]:
def get_wikipedia_headers(url):
    
    response = requests.get(url)
    
    if response.status_code == 200:
       
        soup = BeautifulSoup(response.content, 'html.parser')

       
        headers = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6'])

        
        header_texts = [header.get_text() for header in headers]

        return header_texts
    else:
        print(f"Failed to fetch data. Status Code: {response.status_code}")
        return None

def create_dataframe(headers):
    
    df = pd.DataFrame({'Headers': headers})
    return df

if __name__ == "__main__":
    wikipedia_url = "https://en.wikipedia.org/wiki/Main_Page"


In [122]:
    headers = get_wikipedia_headers(wikipedia_url)

    if headers:
        
        df = create_dataframe(headers)
        print("DataFrame:")
        print(df)
    else:
        print("Unable to fetch headers.")


DataFrame:
                         Headers
0                      Main Page
1           Welcome to Wikipedia
2  From today's featured article
3               Did you know ...
4                    In the news
5                    On this day
6       Today's featured picture
7       Other areas of Wikipedia
8    Wikipedia's sister projects
9            Wikipedia languages


In [123]:
#2
#Write a python program to display IMDB’s Top rated 100 movies’ data (i.e. name, rating, year of release) and make data frame.
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv

In [124]:
def get_page_contents(movies_url):
    response = requests.get(movies_url, headers={"Accept-Language": "en-US"})
    return BeautifulSoup(response.text, "html.parser")

doc = get_page_contents('http://www.imdb.com/chart/top')

In [125]:
movies = doc.find_all('div', {'class':'lister-item-content'})

In [126]:
title = [movie.find('a').text for movie in movies]
release_year = [movie.find('span', {'class':'lister-item-year text-muted unbold'}).text for movie in movies]
audience_rating = [movie.find('div', {'class':'inline-block ratings-imdb-rating'}).text.strip() for movie in movies]
genre = [movie.find('span', {'class':'genre'}).text.strip() for movie in movies]
runtime = [movie.find('span', {'class':'runtime'}).text for movie in movies]

In [127]:
movies_dict = {
    'Title' : title,
    'Release year' : release_year,
    'Audience rating' : audience_rating,
    'Genre' : genre,
    'Runtime' : runtime
}
movies_df = pd.DataFrame(movies_dict)
movies_df

,Title,Release year,Audience rating,Genre,Runtime


In [128]:
movies_df.to_csv('movies.csv', index = None)

In [129]:
#3
#Write a python program to scrape mentioned details from dineout.co.in : i) Restaurant name ii) Cuisine iii) Location iv) Ratings v) Image URL.
import requests
from bs4 import BeautifulSoup

URL = 'https://www.dineout.co.in/'

def get_details(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    restaurants = soup.find_all('h2', class_='restaurant-name')
    cuisines = soup.find_all('span', class_='double-line-ellipsis')
    locations = soup.find_all('span', class_='restnt-loc ellipsis')
    ratings = soup.find_all('div', class_='tbd-rating')
    images = soup.find_all('img', class_='no-img')

    results = []
    for idx, rest in enumerate(restaurants):
        result = {}
        result['Restaurant Name'] = rest.text
        result['Cuisine'] = cuisines[idx].text
        result['Location'] = locations[idx].text
        result['Ratings'] = ratings[idx]['data-original-title']
        result['Image URL'] = images[idx]['data-src']
        results.append(result)

    return results

if __name__ == "__main__":
    results = get_details(URL)
    for detail in results:
        print("-" * 80)
        print(f"Restaurant Name: {detail['Restaurant Name']}")
        print(f"Cuisine:       {detail['Cuisine']}")
        print(f"Location:      {detail['Location']}")
        print(f"Ratings:       {detail['Ratings']}")
        print(f"Image URL:     {detail['Image URL']}\n")

In [130]:
#4
#Write s python program to display list of respected former finance minister of India(i.e. Name , Term of office) from https://presidentofindia.nic.in/former-presidents.htm and make data frame
URL = "http://www.mof.gov.in/aboutus/finministers.php"

In [131]:
def get_fm_details(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    fms = []
    for row in soup.find_all('table')[-1].find_all('tr'):
        name, term = '', ''
        cells = row.find_all('td')
        if len(cells) >= 2:
            name = cells[1].text.strip()
            term = cells[3].text.replace('\xa0', '')[:-4]  # Remove trailing period and spaces
        fms.append({'Finance Minister': name, 'Term of Office': term})
    return fms

if __name__ == "__main__":
    fms = get_fm_details(URL)
    df = pd.DataFrame(fms)
    print(df)

ConnectionError: HTTPConnectionPool(host='www.mof.gov.in', port=80): Max retries exceeded with url: /aboutus/finministers.php (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000226A403B4D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))